In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_rows', 100)

proposal_df = pd.read_excel('data/pycon_2024_proposal.xlsx', dtype={'id': str})
vote_df = pd.read_excel('data/pycon_2024_review.xlsx', dtype={'vote': str, 'proposal_id': str})
vote_df['vote_int'] = vote_df['vote'].astype(int)

#notice int overflow
simple_df = pd.read_excel('data/simple_prompt_gemini_flash_0216.xlsx', dtype={'vote': str, 'proposal_id': str})
simple_df =simple_df.add_prefix('LLM_')
complete_df = pd.read_excel('data/full_prompt_gemini_flash_0216.xlsx', dtype={'vote': str, 'proposal_id': str})
complete_df = complete_df.add_prefix('LLM_')

/usr/local/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
# Get statistics using vote_int (numeric) and vote (string) columns
vote_stats = vote_df.groupby('proposal_id', as_index=False).agg({
    'vote': [
        ('most_common_vote', lambda x: x.mode().iloc[0]),  # Most common vote as string
        ('vote_counts', lambda x: x.value_counts().to_dict())  # Distribution of votes
    ],
    'vote_int': [
        ('mean', 'mean'),     # Average vote
        ('std', 'std'),       # Standard deviation
        ('count', 'count'),   # Number of votes
        ('median', 'median')  # Median vote
    ]
})

# Flatten column names
vote_stats.columns = ['proposal_id' if col[0] == 'proposal_id' 
                     else f'{col[0]}_{col[1]}' for col in vote_stats.columns]

In [4]:
final_df = proposal_df.merge(vote_stats, left_on='id', right_on='proposal_id', how='left')

merge_simple_df = final_df.merge(simple_df, left_on='id', right_on='LLM_proposal_id', how='left')
merge_complete_df = final_df.merge(complete_df, left_on='id', right_on='LLM_proposal_id', how='left')

In [29]:
output_df = final_df.merge(simple_df, left_on='id', right_on='LLM_proposal_id', how='left')\
.merge(complete_df, left_on='id', right_on='LLM_proposal_id', how='left')

output_df.columns = ['id', 'title', 'category', 'duration', 'language', 'abstract',
       'python_level', 'detailed_description', 'outline', 'supplementary',
       'recording_policy', 'slide_link', 'submitter_id', 'created_at',
       'objective', 'cancelled', 'accepted', 'conference', 'last_updated_at',
       'slido_embed_link', 'referring_policy', 'labels', 'first_time_speaker',
       'live_stream_policy', 'hackmd_embed_link', 'living_in_taiwan',
       'attend_in_person', 'proposal_id', 
       'vote_most_common_vote','vote_vote_counts', 'vote_int_mean', 'vote_int_std', 'vote_int_count', 'vote_int_median', 
       'LLM_summary_simple', 'LLM_comment_simple', 'LLM_vote_simple', 'LLM_proposal_id_simple', 
       'LLM_summary_complete', 'LLM_comment_complete', 'LLM_vote_complete','LLM_proposal_id_complete']

output_df = output_df[['id', 'title', 'category', 'duration', 'language', 'abstract',
       'detailed_description', 'outline', 'supplementary', 'objective',
       'vote_most_common_vote','vote_vote_counts', 'vote_int_mean', 'vote_int_std', 'vote_int_count', 'vote_int_median', 
       'LLM_summary_simple', 'LLM_comment_simple', 'LLM_vote_simple', 'LLM_proposal_id_simple', 
       'LLM_summary_complete', 'LLM_comment_complete', 'LLM_vote_complete','LLM_proposal_id_complete']]

In [32]:
#pd.set_option('display.max_columns', 50)

output_df['human_eval'] = ''
output_df.to_excel('data/pycon_2024_proposal_with_llm_and_review_0217.xlsx', index=False)

# confusion matrix

In [10]:
# 修改後的檢查函數
def check_vote_distribution(merged_df):
    print("\n=== Vote Distribution Analysis ===")
    
    # LLM votes distribution
    print("\nLLM Vote Distribution:")
    print(merged_df['LLM_vote'].value_counts(normalize=True).round(3))
    
    # Human votes distribution
    print("\nHuman Vote Distribution:")
    print(merged_df['vote_most_common_vote'].value_counts(normalize=True).round(3))
    
    # Vote agreement analysis
    agreement = (merged_df['LLM_vote'] == merged_df['vote_most_common_vote'])
    print(f"\nOverall Agreement Rate: {agreement.mean():.3f}")
    
    # Disagreement cases analysis
    disagreement_df = merged_df
    if len(disagreement_df) > 0:
        print("\nAgreement Cases Analysis:")
        print(pd.crosstab(
            disagreement_df['LLM_vote'], 
            disagreement_df['vote_most_common_vote'],
            margins=True
        ))

In [15]:
check_vote_distribution(merge_simple_df)


=== Vote Distribution Analysis ===

LLM Vote Distribution:
LLM_vote
+0    0.753
+1    0.221
-1    0.013
-0    0.013
Name: proportion, dtype: float64

Human Vote Distribution:
vote_most_common_vote
+0    0.623
+1    0.182
-0    0.156
-1    0.039
Name: proportion, dtype: float64

Overall Agreement Rate: 0.597

Agreement Cases Analysis:
vote_most_common_vote  +0  +1  -0  -1  All
LLM_vote                                  
+0                     38   8  10   2   58
+1                     10   6   1   0   17
-0                      0   0   1   0    1
-1                      0   0   0   1    1
All                    48  14  12   3   77


In [16]:
check_vote_distribution(merge_complete_df)


=== Vote Distribution Analysis ===

LLM Vote Distribution:
LLM_vote
+0    0.896
+1    0.052
-0    0.052
Name: proportion, dtype: float64

Human Vote Distribution:
vote_most_common_vote
+0    0.623
+1    0.182
-0    0.156
-1    0.039
Name: proportion, dtype: float64

Overall Agreement Rate: 0.597

Agreement Cases Analysis:
vote_most_common_vote  +0  +1  -0  -1  All
LLM_vote                                  
+0                     44  13  11   1   69
+1                      3   1   0   0    4
-0                      1   0   1   2    4
All                    48  14  12   3   77
